In [1]:
import csv

import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [9]:
with open('data\\CleanTwitter.csv', 'r', newline='') as clean:
    read = csv.reader(clean, delimiter=',')
    
    stop_words = set(stopwords.words('english'))
    extended_stops = {"ya", "yo", "yu", "da", "em", "im", "theres", "dat", "dats", "aint", "thats", "doe", "ur"}
    negatives = ["no", "none", "not"]
    
    with open('data\\TwitterPreprocessed.csv', 'w', newline='') as prep:
        write = csv.writer(prep)
        
        prev = -1
        progress = 0
        
        for idx,i in enumerate(read):

            word_tokens = nltk.word_tokenize(i[1])
            filtered_sentence = []
            
            for w in word_tokens:
                if w not in stop_words and w not in extended_stops and len(w) > 1 or w in negatives:
                    filtered_sentence.append(w)
            
            filtered_sentence = [j[0] for j in groupby(filtered_sentence)]
            filtered_sentence = TreebankWordDetokenizer().detokenize(filtered_sentence)
            
            if filtered_sentence:
                write.writerow((i[0], filtered_sentence))

In [10]:
with open('data\\corpus.csv', 'r') as corp:
    read = csv.reader(corp)
    
    stop_words = set(stopwords.words('english'))
    extended_stops = {"ya", "yo", "yu", "da", "em", "im", "theres", "dat", "dats", "aint", "thats", "doe", "ur"}
    negatives = ["no", "none", "not"]
    
    df = []
    
    for idx,i in enumerate(read):
        
        tokens = nltk.word_tokenize(i[1])
        filtered = []
        
        for w in tokens:
            if w not in stop_words and w not in extended_stops and len(w) > 1 or w in negatives:
                filtered.append(w)
        df.append(filtered)
        
    
    with open('data\\allTokens.csv', 'w', newline='') as tok:
        write = csv.writer(tok)
        
        for i in df:
            write.writerow((i))

In [7]:
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath

In [8]:
with open('data\\allTokens.csv', 'r') as wordsList:
    read = csv.reader(wordsList)
    
    words = []
    
    for i in read:
        words.append(i)
    vector_model = Word2Vec(sentences=words, size=300, window=5, min_count=5, workers=4, sg=0)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import *

import numpy as np

In [3]:
with open('data/TwitterPreprocessed.csv', 'r') as dataList:
    read = csv.reader(dataList)
    
    hate = 0
    noHate = 0
    
    for i in read:
        if i[0] == '1':
            hate += 1
        else:
            noHate += 1

with open('data/TwitterPreprocessed.csv', 'r') as dataList:
    read = csv.reader(dataList)
    tokenizer = Tokenizer(num_words=12000, oov_token="<UNK>")
    
    maxlen = 128
    all_data = []
    
    x_train_text = []
    x_train = []
    
    y_train = np.array([])
    y_train = y_train.astype('int64') 
    
    x_test_text = []
    x_test = []
    
    y_test = np.array([])
    y_test = y_test.astype('int64') 
    
    iter1 = 0
    iter2 = 0
    
    for i,j in read:
        if i == '1':
            iter1 += 1
            
            if iter1 <= round(hate*0.8):
                x_train_text.append(j)
                y_train = np.append(y_train, int(i))
            else:
                x_test_text.append(j)
                y_test = np.append(y_test, int(i))
        
        elif i == '0':
            iter2 += 1
            
            if iter2 <= round(noHate*0.8):
                x_train_text.append(j)
                y_train = np.append(y_train, int(i))
            else:
                x_test_text.append(j)
                y_test = np.append(y_test, int(i))
        
        all_data.append(j)
    
    tokenizer.fit_on_texts(all_data)
    word_index = tokenizer.word_index
    
    train_sequences = tokenizer.texts_to_sequences(x_train_text)
    x_train = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=maxlen)
    
    test_sequences = tokenizer.texts_to_sequences(x_test_text)
    x_test = pad_sequences(test_sequences, padding='post', truncating='post', maxlen=maxlen)

In [34]:
import pickle

In [38]:
pickle.dump(word_index, open("pickles\\word_index.pickle", 'wb'))

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout, Bidirectional, Input, Concatenate, Reshape
from tensorflow.keras.layers import LSTM, Conv1D
from tensorflow.keras.layers import GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D
from keras import regularizers

import tensorflow as tf
import keras

Using TensorFlow backend.


In [5]:
def initWeights():
    zero_vec = np.array([])
    zero_vec.astype('float32')

    for i in vector_model["random"]:
        zero_vec = np.append(zero_vec, 0)

    embedding_weights = np.zeros((max_features, dim_count))
    for word,index in word_index.items():
        try:
            embedding_weights[index, :] = vector_model[word]
        except:
            embedding_weights[index, :] = zero_vec
            
    return embedding_weights

In [47]:
dim_count = 300

max_features = len(word_index) + 1
embedding_weights = initWeights()
vocab_len = len(embedding_weights)

embedding_input = Input(shape=(128,), name='embedding_input')

embedding = Embedding(max_features, dim_count, trainable = False, weights = [embedding_weights])(embedding_input)
bilstm = Bidirectional(LSTM(128, dropout=0.4, recurrent_dropout=0.4, return_sequences=True,
                            activity_regularizer=regularizers.l1(0.01), kernel_regularizer=regularizers.l2(0.01)))(embedding)
lstm = LSTM(128, dropout=0.4, recurrent_dropout=0.4, return_sequences=True)(bilstm)
maxPool = GlobalMaxPooling1D()(lstm)
averagePool = GlobalAveragePooling1D()(lstm)
concat = Concatenate()([maxPool, averagePool])
dropout = Dropout(0.2)(concat)
dense = Dense(128, activation='relu')(dropout)
dense = Dense(64, activation='relu')(dense)
out = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=embedding_input, outputs=out)

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), shuffle=True)
model.evaluate(x_test, y_test, batch_size=32, verbose = 2)

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Train on 7141 samples, validate on 1785 samples
Epoch 1/20
7141/7141 [==============================] - 430s 60ms/sample - loss: 3.4164 - accuracy: 0.7944 - precision_2: 0.6972 - recall_2: 0.8522 - val_loss: 0.9362 - val_accuracy: 0.8868 - val_precision_2: 0.8172 - val_recall_2: 0.8321
Epoch 2/20
7141/7141 [==============================] - 421s 59ms/sample - loss: 0.9380 - accuracy: 0.8663 - precision_2: 0.8383 - recall_2: 0.8366 - val_loss: 0.5737 - val_accuracy: 0.9154 - val_precision_2: 0.8591 - val_recall_2: 0.8482
Epoch 3/20
7141/7141 [==============================] - 428s 60ms/sample - loss: 0.7238 - accuracy: 0.8695 - precision_2: 0.8630 - recall_2: 0.8518 - val_loss: 0.5776 - val_accuracy: 0.8852 - val_precision_2: 0.8723 - val_recall_2: 0.8512
Epoch 4/20
7141/7141 [==============================] - 422s 59ms/sample - loss: 0.6525 - accuracy: 0.8765 - precision_2: 0.8753 - recall_2: 0.8525 - val_loss: 0.4695 - val_accuracy: 0.9076 - val_precision_2: 0.8810 - val_recall_2: 0.8

[0.305625716419447, 0.90532213, 0.90446866, 0.8779811]

In [49]:
model.save("NN_models\\BiLSTM-LSTM_Vinf.h5")

In [10]:
dim_count = 300

max_features = len(word_index) + 1
embedding_weights = initWeights()
vocab_len = len(embedding_weights)

model = Sequential()

model.add(Embedding(max_features, dim_count, trainable = False, weights = [embedding_weights]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True,
               activity_regularizer=regularizers.l1(0.001), kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv1D(int(128 / 2), 3, kernel_initializer='he_normal', activation='relu', padding='valid',
                activity_regularizer=regularizers.l1(0.001), kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv1D(int(128 / 4), 3, kernel_initializer='he_normal', activation='relu', padding='valid',
                activity_regularizer=regularizers.l1(0.001), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.4))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), shuffle=True)
model.evaluate(x_test, y_test, batch_size=32, verbose = 2)

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Train on 7141 samples, validate on 1785 samples
Epoch 1/10
7141/7141 [==============================] - 94s 13ms/sample - loss: 0.9108 - accuracy: 0.8566 - precision_2: 0.8123 - recall_2: 0.8478 - val_loss: 0.6484 - val_accuracy: 0.8975 - val_precision_2: 0.8827 - val_recall_2: 0.8718
Epoch 2/10
7141/7141 [==============================] - 87s 12ms/sample - loss: 0.5660 - accuracy: 0.8954 - precision_2: 0.8833 - recall_2: 0.8771 - val_loss: 0.5386 - val_accuracy: 0.8790 - val_precision_2: 0.8963 - val_recall_2: 0.8822
Epoch 3/10
7141/7141 [==============================] - 92s 13ms/sample - loss: 0.4556 - accuracy: 0.9037 - precision_2: 0.8948 - recall_2: 0.8837 - val_loss: 0.3861 - val_accuracy: 0.9244 - val_precision_2: 0.9040 - val_recall_2: 0.8852
Epoch 4/10
7141/7141 [==============================] - 84s 12ms/sample - loss: 0.3957 - accuracy: 0.9109 - precision_2: 0.9064 - recall_2: 0.8863 - val_loss: 0.3687 - val_accuracy: 0.9171 - val_precision_2: 0.9122 - val_recall_2: 0.8886


[0.30614780904699107, 0.9187675, 0.93371445, 0.89681536]